## PDF to txt
libraries = `pip install PyPDF2 pytesseract pillow pdf2image`
* Tesseract ocr needs to be set up and added to PATH

In [11]:
pdfdir = '/Users/calebmasters/Free From Onedrive mess/Project directory/UNFCCC_speeches/speeches/converttxt'
txtdir = '/Users/calebmasters/Free From Onedrive mess/Project directory/UNFCCC_speeches/speeches/txtconvertednew'
txtolddir = '/Users/calebmasters/Free From Onedrive mess/project directory/UNFCCC_speeches/speeches/txtold'

In [13]:
import os
import PyPDF2
from PIL import Image
import pytesseract
from pytesseract import Output
from pdf2image import convert_from_path

def convert_pdf_to_text(path):
    text_content = []
    pdf_file_obj = open(path, 'rb')
    pdf_reader = PyPDF2.PdfReader(pdf_file_obj)

    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        try:
            text = page.extract_text()
            if text:
                text_content.append(text)
            else:
                raise ValueError("No text found, use OCR.")
        except:
            # Fallback to OCR for scanned/image-based PDF pages
            images = convert_from_path(path, first_page=page_num+1, last_page=page_num+1)
            for image in images:
                text = pytesseract.image_to_string(image, lang='eng', output_type=Output.STRING)
                text_content.append(text)

    pdf_file_obj.close()
    return "\n".join(text_content)

# Specify the directory containing PDFs
pdf_directory = pdfdir
output_directory = txtdir

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Convert each PDF to a text file
for pdf_file in os.listdir(pdf_directory):
    if pdf_file.endswith('.pdf'):
        pdf_path = os.path.join(pdf_directory, pdf_file)
        output_text = convert_pdf_to_text(pdf_path)
        text_filename = os.path.join(output_directory, os.path.splitext(pdf_file)[0] + '.txt')
        with open(text_filename, 'w', encoding='utf-8') as text_file:
            text_file.write(output_text)



In [7]:
import os

def find_unconverted_pdfs(pdf_directory, txt_directory):
    # Get a set of PDF filenames without extension
    pdf_files = {os.path.splitext(file)[0] for file in os.listdir(pdf_directory) if file.endswith('.pdf' or '.docx')}
    
    # Get a set of text filenames without extension
    txt_files = {os.path.splitext(file)[0] for file in os.listdir(txt_directory) if file.endswith('.txt' or '.rtf' or '')}
    
    # Find PDF files that don't have a corresponding text file
    unconverted_pdfs = pdf_files - txt_files
    
    return unconverted_pdfs

# Specify the directories containing PDFs and text files
pdf_directory = '/Users/calebmasters/Free From Onedrive mess/Project directory/UNFCCC_speeches/speeches/PDF'
output_directory = '/Users/calebmasters/Free From Onedrive mess/Project directory/UNFCCC_speeches/speeches/txt'

# Find unconverted PDFs
unconverted_pdfs = find_unconverted_pdfs(pdf_directory, output_directory)

# Print or handle the unconverted PDFs
if unconverted_pdfs:
    print("Unconverted PDF files:")
    for pdf in unconverted_pdfs:
        print(pdf)
else:
    print("All PDF files have been converted.")


Unconverted PDF files:
23NANA630408


In [5]:
import os

# Function to get the size of a directory
def get_dir_size(dir):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(dir):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    return total_size

# Function to count the number of files in a directory
def count_files(dir):
    count = 0
    for dirpath, dirnames, filenames in os.walk(dir):
        count += len(filenames)
    return count

def count_directory(directory_path):
    if __name__ == '__main__':
        directory = directory_path
        if not os.path.exists(directory):
            print("Directory does not exist!")
            exit()

        # Get the size of the directory
        dir_size = get_dir_size(directory)
        print(f"Total size of the directory: {dir_size} bytes")

        # Count the number of files in the directory
        file_count = count_files(directory)
        print(f"Total number of files: {file_count}")

count_directory('/Users/calebmasters/Free From Onedrive mess/Project directory/UNFCCC_speeches/speeches/To be translated/French/txt')
count_directory('/Users/calebmasters/Free From Onedrive mess/Project directory/UNFCCC_speeches/speeches/To be translated/French/pdf')



Total size of the directory: 351609 bytes
Total number of files: 67
Total size of the directory: 14194354 bytes
Total number of files: 67


## Copy PDF files

In [9]:
import os
import pandas as pd
import shutil

# Paths
csv_path = '/Users/calebmasters/Free From Onedrive mess/Project directory/UNFCCC_speeches/csv/UNFCCC.csv'  # Update this path to your CSV file location
source_directory = '/Users/calebmasters/Free From Onedrive mess/Project directory/UNFCCC_speeches/speeches/PDF'  # Update this to your source directory
# source_directory = '/Users/calebmasters/Free From Onedrive mess/Project directory/UNFCCC_speeches/speeches/txt'  # Update this to your source directory
destination_directory = '/Users/calebmasters/Free From Onedrive mess/Project directory/UNFCCC_speeches/speeches/Translations/originals all/pdf'  # Update this to your destination directory
# destination_directory = '/Users/calebmasters/Free From Onedrive mess/Project directory/UNFCCC_speeches/speeches/Translations/Other/txt'
# destination_directory = '/Users/calebmasters/Free From Onedrive mess/Project directory/UNFCCC_speeches/speeches/Translations/Other/pdf'



# User input for filtering
column_name = 'Language'
match_value = 'ur' #Leave blank for any

# Read CSV file
try:
    df = pd.read_csv(csv_path)
except FileNotFoundError:
    print(f"CSV file not found: {csv_path}")
    exit()

# Ensure the destination directory exists
try:
    os.makedirs(destination_directory, exist_ok=True)
except FileExistsError:
    print(f"Destination directory already exists: {destination_directory}")
    exit()

# Extensions to check
extensions = ['.pdf', '.docx', '.txt', 'rtf']

# Filter the CSV data
if column_name in df.columns:
    if match_value:
        df = df[df[column_name] == match_value]
    else:
        df = df[df[column_name].notna()]

# Copy files
for index, row in df.iterrows():
    base_file_name = row['File Name']
    found = False

    # Check each extension
    for ext in extensions:
        file_name = f"{base_file_name}{ext}"
        source_path = os.path.join(source_directory, file_name)
        if os.path.exists(source_path):
            try:
                shutil.copy(source_path, destination_directory)
                found = True
            except:
                print(f"Error copying file: {file_name}")
                continue
            break  # Stop checking other extensions if file is found

    if not found:
        print(f"File not found for any expected extensions: {base_file_name}")

print("Files copied successfully.")


Files copied successfully.


## Cutting and moving pdfs removed from dataset


In [4]:

import pandas as pd
import shutil
import os

# Paths
csv_path = '/Users/calebmasters/Free From Onedrive mess/Project directory/UNFCCC_speeches/csv/UNFCCC.csv'  # Update this path to your CSV file location
source_directory = '/Users/calebmasters/Free From Onedrive mess/Project directory/UNFCCC_speeches/speeches/txt'  # Update this to your source directory
destination_directory = '/Users/calebmasters/Free From Onedrive mess/Project directory/UNFCCC_speeches/speeches/Ommitted txt'  # Update this to your destination directory

# Read CSV file
df = pd.read_csv(csv_path)
file_names_from_csv = df['File Name'].apply(lambda x: os.path.splitext(x)[0]).tolist()

# Ensure the destination directory exists
os.makedirs(destination_directory, exist_ok=True)

# Iterate over all files in the source directory
for file in os.listdir(source_directory):
    base_name = os.path.splitext(file)[0]  # Get the base file name without extension

    # Check if the base name of the file is not in the list from the CSV
    if base_name not in file_names_from_csv:
        source_path = os.path.join(source_directory, file)
        destination_path = os.path.join(destination_directory, file)

        # Copy and then delete the file
        shutil.copy(source_path, destination_path)
        os.remove(source_path)
        print(f"Copied and deleted: {file}")
    else:
        print()

print("Operation completed successfully.")












































































































































































































































































Copied and deleted: COP27 Botswana.txt























































































































































































































































































































































































































































































































































































































































































Operation completed successfully

## Combining PDF's

20mb limit

In [9]:
import os
import io
from PyPDF2 import PdfReader, PdfWriter

def combine_pdfs_with_size_limit(directory_path, max_size_mb=20):
    # Initialize variables
    max_size = max_size_mb * 1024 * 1024  # Convert MB to bytes
    output_index = 1
    writer = PdfWriter()

    # Function to write and reset the writer
    def write_and_reset(writer, index):
        output_filename = f'combined_{index}.pdf'
        with open(output_filename, 'wb') as output_pdf:
            writer.write(output_pdf)
        return PdfWriter(), index + 1

    # List all PDF files
    files = [f for f in os.listdir(directory_path) if f.endswith('.pdf')]
    files.sort()  # Optional: Sort files if needed

    for file_name in files:
        file_path = os.path.join(directory_path, file_name)
        
        # Read the PDF file
        reader = PdfReader(file_path)
        
        # Temporary memory buffer to estimate the file size
        temp_buffer = io.BytesIO()

        # Add pages individually and write to the buffer to estimate size
        for page in reader.pages:
            writer.add_page(page)
            writer.write(temp_buffer)

            # Check if the file size exceeds the limit
            if temp_buffer.tell() > max_size:
                # Finalize the current PDF and start a new one
                writer, output_index = write_and_reset(writer, output_index)
                # Reset buffer
                temp_buffer = io.BytesIO()
                # Add the current page to the new writer
                writer.add_page(page)
                writer.write(temp_buffer)

    # Write the last set of combined PDFs if any pages have been added
    if len(writer.pages) > 0:
        write_and_reset(writer, output_index)

# Usage example
directory_path = '/Users/calebmasters/Free From Onedrive mess/Project directory/UNFCCC_speeches/speeches/To be translated/Arabic'
combine_pdfs_with_size_limit(directory_path)



In [11]:
import os
import io
from PyPDF2 import PdfReader, PdfWriter

def combine_pdfs_with_size_limit(directory_path, max_size_mb=20):
    # Initialize variables
    max_size = max_size_mb * 1024 * 1024  # Convert MB to bytes
    output_index = 1
    writer = PdfWriter()
    current_file_list = []  # List to keep track of files in the current combination

    # Function to write and reset the writer
    def write_and_reset(writer, index, file_list):
        output_filename = f'combined_{index}.pdf'
        with open(output_filename, 'wb') as output_pdf:
            writer.write(output_pdf)
        print(f"Created '{output_filename}' with files: {file_list}")
        return PdfWriter(), index + 1, []

    # List all PDF files
    files = [f for f in os.listdir(directory_path) if f.endswith('.pdf')]
    files.sort()  # Optional: Sort files if needed

    for file_name in files:
        file_path = os.path.join(directory_path, file_name)
        
        # Read the PDF file
        reader = PdfReader(file_path)
        
        # Temporary memory buffer to estimate the file size
        temp_buffer = io.BytesIO()

        # Add pages individually and write to the buffer to estimate size
        for page in reader.pages:
            writer.add_page(page)
            writer.write(temp_buffer)

            # Check if the file size exceeds the limit
            if temp_buffer.tell() > max_size:
                # Finalize the current PDF and start a new one
                writer, output_index, current_file_list = write_and_reset(writer, output_index, current_file_list)
                # Reset buffer
                temp_buffer = io.BytesIO()
                # Add the current page to the new writer
                writer.add_page(page)
                writer.write(temp_buffer)

        # Add the file to the current list after checking size
        current_file_list.append(file_name)

    # Write the last set of combined PDFs if any pages have been added
    if len(writer.pages) > 0:
        write_and_reset(writer, output_index, current_file_list)

# Usage example
directory_path = '/Users/calebmasters/Free From Onedrive mess/Project directory/UNFCCC_speeches/speeches/To be translated/Arabic'
combine_pdfs_with_size_limit(directory_path)


Created 'combined_1.pdf' with files: ['18AlgeriaCOP24186330.pdf', '18KuwaitCOP24186437.pdf', '18OmanCOP24185051.pdf', '18TunisiaCOP24186404.pdf', '19AlgeriaCOP25204579.pdf', '19BahrainCOP25204114.pdf', '19KuwaitCOP25204061.pdf']
Created 'combined_2.pdf' with files: ['19MoroccoCOP25204069.pdf', '21AlgeriaCOP26310802.pdf', '21KuwaitCOP26309209.pdf', '21NACOP26309286.pdf', '21OmanCOP26310839.pdf']
Created 'combined_3.pdf' with files: ['21QatarCOP26310872.pdf', '21SyriaCOP26310880.pdf', '21TunisiaCOP26310884.pdf', '22AlgeriaCOP27622747.pdf', '22BahrainCOP27623882.pdf', '22LebanonCOP27624199.pdf', '22NACOP27623385.pdf', '22NACOP27624084.pdf', '22OmanCOP27623899.pdf', '22QatarCOP27623900.pdf', '22SudanCOP27624198.pdf', '22YemenCOP27623027.pdf', '23BahrainCOP28636641.pdf', '23KyrgyzstanCOP28636282.pdf']
Created 'combined_4.pdf' with files: ['23LibyaCOP28636285.pdf', '23NACOP28634579.pdf', '23NACOP28635860.pdf', '23NACOP28636637.pdf', '23OmanCOP28636628.pdf', '23QatarCOP28636633.pdf', '23Syria

Created 'combined_1.pdf' with files: ['18AlgeriaCOP24186330.pdf', '18KuwaitCOP24186437.pdf', '18OmanCOP24185051.pdf', '18TunisiaCOP24186404.pdf', '19AlgeriaCOP25204579.pdf', '19BahrainCOP25204114.pdf', '19KuwaitCOP25204061.pdf']
Created 'combined_2.pdf' with files: ['19MoroccoCOP25204069.pdf', '21AlgeriaCOP26310802.pdf', '21KuwaitCOP26309209.pdf', '21NACOP26309286.pdf', '21OmanCOP26310839.pdf']
Created 'combined_3.pdf' with files: ['21QatarCOP26310872.pdf', '21SyriaCOP26310880.pdf', '21TunisiaCOP26310884.pdf', '22AlgeriaCOP27622747.pdf', '22BahrainCOP27623882.pdf', '22LebanonCOP27624199.pdf', '22NACOP27623385.pdf', '22NACOP27624084.pdf', '22OmanCOP27623899.pdf', '22QatarCOP27623900.pdf', '22SudanCOP27624198.pdf', '22YemenCOP27623027.pdf', '23BahrainCOP28636641.pdf', '23KyrgyzstanCOP28636282.pdf']
Created 'combined_4.pdf' with files: ['23LibyaCOP28636285.pdf', '23NACOP28634579.pdf', '23NACOP28635860.pdf', '23NACOP28636637.pdf', '23OmanCOP28636628.pdf', '23QatarCOP28636633.pdf', '23SyriaCOP28635631.pdf', '23TunisiaCOP28636639.pdf', '24KuwaitCOP28637243.pdf', '24SaudiArabiaCOP28637245.pdf']

## Make pdfs translateable txt files

In [16]:
import fitz  # PyMuPDF
from PIL import Image, ImageEnhance, ImageFilter
import pytesseract
import io

# Enhance image for better OCR recognition
def enhance_image(image):
    # Convert to grayscale
    image = image.convert('L')
    # Increase contrast
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(2.0)
    # Apply sharpening
    image = image.filter(ImageFilter.SHARPEN)
    return image

def extract_text_from_pdf_image(pdf_path):
    doc = fitz.open(pdf_path)
    full_text = []
    
    for page in doc:
        image_list = page.get_images(full=True)
        
        for img_index, img in enumerate(image_list):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            
            # Convert image bytes to PIL Image and enhance it
            image = Image.open(io.BytesIO(image_bytes))
            image = enhance_image(image)
            
            # Use pytesseract to do OCR on the enhanced image
            text = pytesseract.image_to_string(image, lang='ara+eng', config='--psm 6')
            full_text.append(text)
    
    doc.close()
    return full_text

# Example usage
pdf_path = '/Users/calebmasters/Free From Onedrive mess/Project directory/UNFCCC_speeches/speeches/To be translated/Arabic/18AlgeriaCOP24186330.pdf'
text_content = extract_text_from_pdf_image(pdf_path)
print("\n".join(text_content))



ا
د



# Change naming scheme

In [ ]:
import pandas as pd
import os
import shutil

# Load CSV data
csv_path = '/Users/calebmasters/Free From Onedrive mess/Project directory/UNFCCC_speeches/csv/Subsets/UNFCCCcountries.csv'
df = pd.read_csv(csv_path)

# Define the renaming schema
rename_schema = [
    {'column': 'classification', 'replace': {'Country_Speech': 'cs'}, 'exact': False},
    
    # {'column': 'Organisation/Speaker', 'replace': {}, 'exact': True},  # This column's data will be copied exactly
    # Add more mappings here as needed
]

# Directory for renamed files
new_directory = 'path/to/new_directory'
os.makedirs(new_directory, exist_ok=True)

def construct_new_filename(row):
    parts = []
    for schema in rename_schema:
        part = row[schema['column']]
        if not schema['exact']:
            for key, value in schema['replace'].items():
                part = part.replace(key, value)
        parts.append(part)
    return "_".join(parts)

# Function to find the actual file regardless of extension
def find_file(base_path, name):
    for filename in os.listdir(base_path):
        if os.path.splitext(filename)[0] == name:
            return filename
    return None

# Process each file
base_path = 'path/to/original_files'  # Update with the correct path
for index, row in df.iterrows():
    old_name = row['Name of Speech']
    file_found = find_file(base_path, old_name)
    if file_found:
        new_name = construct_new_filename(row) + os.path.splitext(file_found)[1]  # Keep original extension
        old_path = os.path.join(base_path, file_found)
        new_path = os.path.join(new_directory, new_name)
        shutil.copy2(old_path, new_path)  # Copy and rename the file
        df.at[index, 'New_Name'] = new_name  # Add new name to the DataFrame
    else:
        print(f"File corresponding to {old_name} not found.")

# Save the updated DataFrame back to CSV
updated_csv_path = 'path/to/your/updated_csvfile.csv'
df.to_csv(updated_csv_path, index=False)

# Note: The script now allows for copying exact column data as part of the new filename.
